In [5]:
# channels = ["WW","bb","tautau","mumu","nuenue","numunumu","nutaunutau"]
channels = ["numunumu"]
# channels = ["WW","numunumu", 'bb']
# channels = ["WW","numunumu"]
# channels = ["numunumu","bb", "WW"]
# channels = ["nuenue", "numunumu", "nutaunutau"]


# profile = ['NFW','Burkert']
profile = ['NFW']



In [6]:

cfig_file = '/home/tchau/code/condor_submit/UL/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=100'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/UL/UL.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/UpperLimit_LLH_Interval.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 6GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) --process $(proc)  -u $(u) -l $(l) -n $(n) --mc $(mc) -b $(bkg) --nsample $(nsample)'+'\n')
    s.write('queue')
    s.close()

In [11]:
import os
submit = '/home/tchau/code/condor_submit/UL/submit.sh'
n = 30
# n = 5
# nsample = 500
nsample = 0

# bkgs=['FFT']
bkgs=['precomp']

# mcs=['0000', '1122']
# mcs = ['0000', '1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133', '1122_disCSMS', '1122_disCSMSx3', '1122_disCSMSx-3']
# mcs = ['1122']
# mcs = ['1122','1122_disCSMS', '1122_disCSMSx3', '1122_disCSMSx-3']
mcs = ['1122_disCSMSx3', '1122_disCSMSx-3', '1122_disCSMS']
prcs = ["ann", "decay"]
# prcs = ["decay"]

path = '/home/tchau/code/condor_submit/UL/'
if not (os.path.exists(path)): os.makedirs(path)
dag_file = '{}/UL_{}trial.dag'.format(path, nsample)

with open(dag_file, 'w') as f:
    for bkg in bkgs:
        for mc in mcs:
            for c in channels:
                for p in profile:
                    for prc in prcs:
                        if prc=='ann':
                            masses = {"WW":[90, 8000], "bb":[15, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}
                        elif prc=='decay':    
                            masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[5, 400],"numunumu":[5, 400],"nutaunutau":[5,400]}
                        # masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[1.5, 5],"numunumu":[1.5, 5],"nutaunutau":[1.5,5]}

                            f.write('JOB UL_{}_{}_{}_{}_{}_{}_{}trial'.format(prc, c,p, n, mc, bkg, nsample)+' UL.submit \n')
                            f.write('VARS UL_{}_{}_{}_{}_{}_{}_{}trial'.format(prc, c , p, n, mc, bkg, nsample)+' JOBNAME="UL_{}_{}_{}_{}_{}_{}_{}trial"'.format(prc, c, p, n, mc, bkg, nsample) +' c="{}" p="{}" proc="{}" u="{}" l="{}" n="{}" mc="{}" bkg="{}" nsample="{}"'.format(c, p, prc, masses[c][1], masses[c][0], n, mc, bkg, nsample)+'\n')
f.close()
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
    print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL//UL_0trial.dag



# <font color="darkmagenta"> With options on error for Jfactor </font>

In [2]:

cfig_file = '/home/tchau/code/condor_submit/UL/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/UL/UL_errorJ.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/UpperLimit_LLH_Interval.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 6GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) -u $(u) -l $(l) -n $(n) --mc $(mc) -b $(bkg) --nsample $(nsample) --errorJ $(errorJ) --process $(proc)'+'\n')

    s.write('queue')
    s.close()

In [3]:
import os
submit = '/home/tchau/code/condor_submit/UL/submit.sh'
n = 30
nsample = 0
bkgs=['precomp']
mcs=['1122']
prcs = ["ann", "decay"]

# mcs = ['0000', '1122', '1124', '÷1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133']
errorJ=['errors1', 'errors2']
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for bkg in bkgs:
        for mc in mcs:
            for c in channels:
                for p in profile:
                    for error in errorJ:
                        for prc in prcs:
                            if prc=='ann':
                                masses = {"WW":[90, 8000], "bb":[15, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}
                            elif prc=='decay':    
                                masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[5, 400],"numunumu":[5, 400],"nutaunutau":[5,400]}
                            path = '/home/tchau/code/condor_submit/UL/{}_{}_{}_{}_{}_{}_{}trial_Jfactor{}'.format(prc, c, p, n, mc, bkg, nsample, error)
                            if not (os.path.exists(path)): os.makedirs(path)
                            dag_file = '{}/{}_{}_{}_{}_{}_{}_{}trial_J{}.dag'.format(path, prc, c, p, n, bkg, mc, nsample, error)
                            with open(dag_file, 'w') as f:
                                f.write('JOB UL_{}_{}_{}_{}_{}_{}_{}trial_J{}.dag'.format(prc, c, p, n, mc, bkg, nsample, error)+' UL_errorJ.submit \n')
                                f.write('VARS UL_{}_{}_{}_{}_{}_{}_{}trial_J{}.dag'.format(prc, c, p, n, mc, bkg, nsample, error)+' JOBNAME="UL_{}_{}_{}_{}_{}_{}trial_J{}"'.format(c, p, n, mc, bkg, nsample, error) +' c="{}" p="{}" u="{}" l="{}" n="{}" mc="{}" bkg="{}" nsample="{}" errorJ="{}" proc="{}"'.format(c, p, masses[c][1], masses[c][0], n, mc, bkg, nsample, error, prc)+'\n')
                            f.close()
                            s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
                            print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/ann_WW_NFW_30_1122_precomp_0trial_Jfactorerrors1/ann_WW_NFW_30_precomp_1122_0trial_Jerrors1.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/decay_WW_NFW_30_1122_precomp_0trial_Jfactorerrors1/decay_WW_NFW_30_precomp_1122_0trial_Jerrors1.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/ann_WW_NFW_30_1122_precomp_0trial_Jfactorerrors2/ann_WW_NFW_30_precomp_1122_0trial_Jerrors2.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/decay_WW_NFW_30_1122_precomp_0trial_Jfactorerrors2/decay_WW_NFW_30_precomp_1122_0trial_Jerrors2.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/ann_WW_Burkert_30_1122_precomp_0trial_Jfactorerrors1/ann_WW_Burkert_30_precomp_1122_0trial_Jerrors1.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/decay_WW_Burkert_30_1122_precomp_0trial_Jfactorerrors1/

# With and without GC astro flux included

In [7]:

cfig_file = '/home/tchau/code/condor_submit/UL/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/UL/UL_GP.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/UpperLimit_LLH_Interval.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 6GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) -u $(u) -l $(l) -n $(n) --mc $(mc) -b $(bkg) --process $(proc) --nsample $(nsample) --gcinj $(gcinj) --gcmodel $(gcmodel)'+'\n')

    s.write('queue')
    s.close()

In [8]:
import os
submit = '/home/tchau/code/condor_submit/UL/submit.sh'
n = 30
nsample = 0
bkgs=['precomp']
# mcs=['0000', '1122']
# mcs = ['0000', '1122', '1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133']
mcs=['1122']

process=['ann']
gcinjs = ['pi0_IC']
# gcmodels = ['pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']
gcmodels = ['KRA50_IC']

path = '/home/tchau/code/condor_submit/UL/'
if not (os.path.exists(path)): os.makedirs(path)
dag_file = '{}/UL_GP_{}trial.dag'.format(path, nsample)


with open(dag_file, 'w') as f:
    for bkg in bkgs:
        for mc in mcs:
            for c in channels:
                for p in profile:
                    for proc in process:
                        for gcinj in gcinjs:
                            for gcmodel in gcmodels:
                                if proc=='ann':
                                    masses = {"WW":[90, 8000], "bb":[15, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}
                                elif proc=='decay':    
                                    masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[5, 400],"numunumu":[5, 400],"nutaunutau":[5,400]}

                                f.write('JOB UL_{}_{}_{}_{}_{}_{}_{}trial_gcinj{}_gcmodel{}'.format(c,p, n, mc, bkg, proc, nsample, gcinj, gcmodel)+' UL_GP.submit \n')
                                f.write('VARS UL_{}_{}_{}_{}_{}_{}_{}trial_gcinj{}_gcmodel{}'.format(c,p, n, mc, bkg, proc, nsample, gcinj, gcmodel)+' JOBNAME="UL_{}_{}_{}_{}_{}_{}_{}trial_gcinj{}_gcmodel{}"'.format(c, p, n, mc, bkg, proc, nsample, gcinj, gcmodel) +' c="{}" p="{}" u="{}" l="{}" n="{}" mc="{}" bkg="{}" proc="{}" nsample="{}" gcinj="{}" gcmodel="{}"'.format(c, p, masses[c][1], masses[c][0], n, mc, bkg, proc, nsample, gcinj, gcmodel)+'\n')
f.close()

with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
    print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL//UL_GP_0trial.dag

